In [14]:
import os

In [15]:
%pwd

'/home/neji/End-to-End-Chest-Cancer-Classification-using-MLflow-DVC'

In [16]:
if "End-to-End-Chest-Cancer-Classification-using-MLflow-DVC" != os.path.basename(os.getcwd()):
    os.chdir("../")

In [17]:
%pwd

'/home/neji/End-to-End-Chest-Cancer-Classification-using-MLflow-DVC'

In [18]:
os.environ["MLFLOW_TRACKING_URI"]="http://localhost:5000"
os.environ["MLFLOW_TRACKING_USERNAME"]="entbappy"
os.environ["MLFLOW_TRACKING_PASSWORD"]="6824692c47a35333f9eac5b10041d5c8edbcef0"

In [19]:
import tensorflow as tf

In [20]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [21]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [22]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [23]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
            mlflow_uri="http://localhost:5000",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config




In [24]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [25]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)


    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        print(f"\n\nMLflow URI: {self.config.mlflow_uri}")
        print(f"Tracking Store Type: {tracking_url_type_store}")
        print(f"Model Path: {self.config.path_of_model}\n\n")
        # print(f"Model Save Format: {keras_model_kwargs}")
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                model = tf.keras.models.clone_model(self.model)
                mlflow.keras.log_model(model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")


In [26]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-01-02 09:29:44,187: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-01-02 09:29:44,194: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-02 09:29:44,197: INFO: common: created directory at: artifacts]
Found 102 images belonging to 2 classes.


2025-01-02 09:29:44.726159: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


7/7 [==============================] - 13s 1s/step - loss: 15.5852 - accuracy: 0.5686
[2025-01-02 09:29:57,299: INFO: common: json file saved at: scores.json]


MLflow URI: http://localhost:5000
Tracking Store Type: http
Model Path: artifacts/training/model.h5




2025/01/02 09:29:57 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-01-02 09:29:57,826: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
[2025-01-02 09:30:00,407: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 13). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: /tmp/tmpplf5kn7t/model/data/model/assets
[2025-01-02 09:30:00,852: INFO: builder_impl: Assets written to: /tmp/tmpplf5kn7t/model/data/model/assets]


2025/01/02 09:30:09 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2025/01/02 09:30:09 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 2


🏃 View run upset-jay-882 at: http://localhost:5000/#/experiments/0/runs/86334ad4b8264fecb33881f2ce39c2b5
🧪 View experiment at: http://localhost:5000/#/experiments/0


Created version '2' of model 'VGG16Model'.
